![](example_content/undatasio_example.png)

In [1]:
# install undatasio
!pip install -U -q undatasio

In [2]:
!conda install -c conda-forge python-dotenv -y -q

Retrieving notices: ...working... done
Solving environment: ...working... done

# All requested packages already installed.



In [3]:
import os
from dotenv import load_dotenv

load_dotenv('.env')

True

In [4]:
UNDATASIO_API_KEY=os.getenv("UNDATASIO_API_KEY")

In [6]:
from undatasio.undatasio import UnDatasIO

undatasio_obj = UnDatasIO(UNDATASIO_API_KEY, task_name='demo')